## 예술의전당 콘서트홀 가격 모델

GOAL : 예술의전당 콘서트홀의 좌석별 위치적 특성을 나타내는 파생변수를 생성

0. 자료 조사 : '예술의전당 콘서트홀 좌석배치도 (2023, 2018)', '예술의전당 콘서트홀 도면 (나라장터 2023)' 
1. 데이터 라벨링 : AutoCAD, excel을 이용해서 좌석별 (x, y, z) 좌표 라벨링
2. 데이터 라벨링 결과물 확인 : describe, scatter plot 시각화
3. 좌표 단위 수정 : mm => cm, round
4. 파생변수 생성 : '거리', '시야각' 등의 최종적으로 필요한 변수

### 1. 라이브러리 로드

In [206]:
# base
import pandas as pd
import numpy as np

# files
from glob import glob

# visualization
import matplotlib.pyplot as plt
import koreanize_matplotlib
import plotly.express as px
import seaborn as sns

In [207]:
# settings
%matplotlib inline
pd.get_option("display.max_columns", 100)
pd.set_option('display.max_rows', 500)


### 2. 데이터셋 로드

In [208]:
# 좌석좌표 엑셀 파일 로드
print(f"""Found.. {len(glob('*.xlsx'))} file(s) : {glob('*.xlsx')}
Reading.. {glob('*.xlsx')[-1]}
""")
df = pd.read_excel(glob('*.xlsx')[-1])

# shape, 데이터 확인
print(df.shape)
df.head()

Found.. 2 file(s) : ['~$예술의전당 콘서트홀 좌석좌표.xlsx', '예술의전당 콘서트홀 좌석좌표.xlsx']
Reading.. 예술의전당 콘서트홀 좌석좌표.xlsx

(2505, 8)


,층,블록,열,넘버,전체_좌석,X,Y,Z
0,1층,A블록,1,1,1층 A블록1열 1,14512.2087,5422.5042,60874
1,1층,A블록,1,2,1층 A블록1열 2,14059.5465,5548.9833,60874
2,1층,A블록,1,3,1층 A블록1열 3,13606.8844,5675.4624,60874
3,1층,A블록,1,4,1층 A블록1열 4,13154.2222,5801.9416,60874
4,1층,A블록,1,5,1층 A블록1열 5,12695.7535,5906.1735,60874


In [209]:
# 무대 중앙을 원점(0, 0, 0)으로 설정
# 엑셀파일의 좌표는 건축도면을 기준으로 되어 있음. > Z좌표에서 무대의 바닥 높이 만큼 빼준다.
df['Z'] = (df['Z'] - 61800)

# # X, Y, Z 좌표는 기존에 float으로 되어 있음 (mm 기준) => int (cm 기준)으로 변경
df['X'] = (round(df['X'], -1) / 10).astype(int)
df['Y'] = (round(df['Y'], -1) / 10).astype(int)
df['Z'] = (round(df['Z'], -1) / 10).astype(int)

df.head()

,층,블록,열,넘버,전체_좌석,X,Y,Z
0,1층,A블록,1,1,1층 A블록1열 1,1451,542,-93
1,1층,A블록,1,2,1층 A블록1열 2,1406,555,-93
2,1층,A블록,1,3,1층 A블록1열 3,1361,568,-93
3,1층,A블록,1,4,1층 A블록1열 4,1315,580,-93
4,1층,A블록,1,5,1층 A블록1열 5,1270,591,-93


In [210]:
# 대칭인 좌석 좌표 개수 확인
temp = df.copy()
temp['|X|'] = abs(temp['X'])
temp = temp.groupby(['|X|', 'Y', 'Z']).count()['X'].reset_index()


print(f"({(temp.loc[temp['X']==2].shape[0])} * 2) + {temp.loc[temp['X']==1].shape[0]} =", end=" ")
print((temp.loc[temp['X']==2].shape[0])*2 + temp.loc[temp['X']==1].shape[0])

(1244 * 2) + 17 = 2505


In [214]:
def get_point(X, Y, Z):
    try:
        return df.loc[(df.X == X) & (df.Y==Y) & (df.Z==Z), '전체_좌석'].iloc[0]
    except:
        return 0
    
df['대칭점'] = df.apply(lambda r: get_point(-r['X'], r['Y'], r['Z']), axis=1)
# df.to_csv('좌석좌표_최종(대칭점 포함).csv', index=False)

,층,블록,열,넘버,전체_좌석,X,Y,Z,대칭점
0,1층,A블록,1,1,1층 A블록1열 1,1451,542,-93,1층 E블록1열 9
1,1층,A블록,1,2,1층 A블록1열 2,1406,555,-93,1층 E블록1열 8
2,1층,A블록,1,3,1층 A블록1열 3,1361,568,-93,1층 E블록1열 7
3,1층,A블록,1,4,1층 A블록1열 4,1315,580,-93,1층 E블록1열 6
4,1층,A블록,1,5,1층 A블록1열 5,1270,591,-93,1층 E블록1열 5
...,...,...,...,...,...,...,...,...,...
2500,2층,BOX6,없음,2,2층 BOX6 2,-1432,505,383,2층 BOX1 1
2501,2층,BOX6,없음,3,2층 BOX6 3,-1500,565,428,2층 BOX1 4
2502,2층,BOX6,없음,4,2층 BOX6 4,-1454,617,428,2층 BOX1 3
2503,2층,BOX6,없음,5,2층 BOX6 5,-1522,677,483,2층 BOX1 6


### 3. 좌석 좌표 시각화

In [5]:
# 무대(원점) 추가
temp = df.iloc[1].copy()
for i in temp.keys():
    if i in ('X', 'Y', 'Z'):
        temp[i] = 0
    else : temp[i] = '무대'
temp = pd.DataFrame(temp).T
temp = pd.concat([df, temp], ignore_index=True)

# 필터
# temp = temp.loc[temp['층'] == '1층']

In [355]:
# 2D scatter plot
px.scatter(temp, x='X', y='Y', color='층', width=600, height=600, hover_name='전체_좌석')

In [356]:
# 3d scatter plot
fig = px.scatter_3d(temp, x='X', y='Y', z='Z', color='층', size_max=18, hover_name='전체_좌석')
fig.update_layout(width=800, height=600)
fig.update_traces(marker={'size': 1})
fig.show()

### 4. 파생변수 생성


#### 4-1) 좌석밀집도 (복도 자리 여부)

In [357]:
# 양 끝 자리 찾기
max_nums = df.groupby(['층', '블록', '열'])['넘버'].transform('max')
min_nums = df.groupby(['층', '블록', '열'])['넘버'].transform('min')

# 끝 열 찾기
max_row = df.groupby(['층', '블록'])['열'].transform('max')

In [358]:
# 자리가 복도 자리인 경우 : 층, 블록, 열 별로 가장 끝 번호인 경우 찾기\
df['복도'] = ((df['넘버'] == min_nums) | (df['넘버'] == max_nums)).astype(int)

# 열이 복도와 인접한 경우 : 첫 열, 끝 열 찾기
cond1 = ((df['열']==max_row) | (df['열']==1))
cond2 = ((df['층']=='1층') & (df['블록'].isin(['A블록', 'E블록'])) & (df['열']==13))
cond3 = ((df['층']=='1층') & (df['블록'].isin(['B블록', 'C블록', 'D블록'])) & (df['열']==14))
df.loc[cond1 | cond2 | cond3, '복도'] += 1

# 2d scatter plot
fig = px.scatter(df, x='X', y='Y', color='복도', facet_col='층', hover_name='전체_좌석')
fig.update_layout(width=1500, height=600)
fig.update_traces(marker={'size': 3})
fig.show()

# 합창석 : FH블록 3열
# 2층 : BD블록 7열
# 3층 : BCDEF블록 1열

#### 4-2) 무대-객석 거리

In [359]:
df['무대객석거리'] = df.apply(
    lambda r: np.sqrt(r['X']**2 + r['Y']**2 + r['Z']**2), axis=1).astype(int)

display(df.head())

# 3d scatter plot
fig = px.scatter_3d(df, x='X', y='Y', z='Z', color='무대객석거리', hover_name='전체_좌석')
fig.update_layout(width=800, height=600)
fig.update_traces(marker={'size': 1})
fig.show()

,층,블록,열,넘버,전체_좌석,X,Y,Z,복도,무대객석거리
0,1층,A블록,1,1,1층 A블록1열 1,1451,542,-93,2,1551
1,1층,A블록,1,2,1층 A블록1열 2,1406,555,-93,1,1514
2,1층,A블록,1,3,1층 A블록1열 3,1361,568,-93,1,1477
3,1층,A블록,1,4,1층 A블록1열 4,1315,580,-93,1,1440
4,1층,A블록,1,5,1층 A블록1열 5,1270,591,-93,1,1403


#### 4-3) 무대정면과의각도
cos 값으로 -1~1 사이로 환산

In [371]:
np.degrees(np.arctan2(df['X'], df['Y'], df['Z']))

0       69.517555
1       68.459024
2       67.347261
3       66.199420
4       65.044882
          ...    
2500   -70.574617
2501   -69.360281
2502   -67.006224
2503   -66.020049
2504   -68.299463
Name: Z, Length: 2505, dtype: float64

In [361]:
df['무대정면과의각도'] = round(np.cos(np.arctan2(df['X'], df['Y'])), 3)
print(df['무대정면과의각도'].describe())

# 2D scatter plot
px.scatter(df, x='X', y='Y', color='무대정면과의각도', width=600, height=600, hover_name='전체_좌석',
           color_continuous_scale=px.colors.diverging.balance)

count    2505.000000
mean        0.687883
std         0.497040
min        -1.000000
25%         0.704000
50%         0.864000
75%         0.965000
max         1.000000
Name: 무대정면과의각도, dtype: float64


4-4)


In [348]:
df['무대정면_상하각도'] = (round(np.rad2deg(np.arcsinh(df['Z'] / df['무대객석거리'])), 1))
df

,층,블록,열,넘버,전체_좌석,X,Y,Z,복도,무대객석거리,무대정면과의각도,상하시야각,무대정면_상하각도
0,1층,A블록,1,1,1층 A블록1열 1,1451,542,-93,2,1551,0.350,-3.4,-3.4
1,1층,A블록,1,2,1층 A블록1열 2,1406,555,-93,1,1514,0.367,-3.5,-3.5
2,1층,A블록,1,3,1층 A블록1열 3,1361,568,-93,1,1477,0.385,-3.6,-3.6
3,1층,A블록,1,4,1층 A블록1열 4,1315,580,-93,1,1440,0.404,-3.7,-3.7
4,1층,A블록,1,5,1층 A블록1열 5,1270,591,-93,1,1403,0.422,-3.8,-3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2500,2층,BOX6,없음,2,2층 BOX6 2,-1432,505,383,1,1565,0.333,13.9,13.9
2501,2층,BOX6,없음,3,2층 BOX6 3,-1500,565,428,1,1659,0.352,14.6,14.6
2502,2층,BOX6,없음,4,2층 BOX6 4,-1454,617,428,1,1636,0.391,14.8,14.8
2503,2층,BOX6,없음,5,2층 BOX6 5,-1522,677,483,1,1734,0.406,15.8,15.8


In [41]:
def 무대정면과의각도(x, y):
    # 무대의 정면(y축)에서 얼마나 벗어나 있는지를 cos값으로 리턴 (-1 ~ 1 사이의 값)
    degree = np.arctan2(x, y)
    return np.cos(np.deg2rad(degree))

def 무대와의거리(x, y, z):
    # 무대 중심(원점 0, 0, 0)에서부터 객석까지의 거리
    return int(round(np.sqrt(x**2 + y**2 + z**2)))

def 객석의각도(x, y):
    




-0.1736481776669303